# Using dynamic RNN and LSTM with TFLite converter

This test script highlights several tricks you need in order to use Dynamic RNN and LSTM in TFLite.
Hopefully this can save your time figuring out these experimental features!

The major part of this script takes the reference from: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/experimental/examples/lstm/unidirectional_sequence_lstm_test.py

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.python.ops import control_flow_util
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.tools import optimize_for_inference_lib
from tensorflow.lite.python.op_hint import convert_op_hints_to_stubs

# Turn warning off
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
print (tf.__version__)

1.13.0-dev20190227


### set up mnist and initial parameters

In [3]:
# download and process mnist
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
# Define constants
# Unrolled through 28 time steps
time_steps = 28
# Rows of 28 pixels
n_input = 28
# Learning rate for Adam optimizer
learning_rate = 0.001
# MNIST is meant to be classified in 10 classes(0-9).
n_classes = 10
# Batch size
batch_size = 16
# Lstm Units.
num_units = 16
TRAIN_STEPS = 1

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### Functions for building the network

#### Trick 1: 
Use `tf.lite.experimental.nn.TFLiteLSTMCell` for LSTM and `tf.lite.experimental.nn.dynamic_rnn` for dynamic rnn

In [4]:
def buildLstmLayer():
    return tf.nn.rnn_cell.MultiRNNCell([
        tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, use_peepholes=True, forget_bias=0, name="rnn1"),
        tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, num_proj=8, forget_bias=0, name="rnn2"),
        tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units // 2,
            use_peepholes=True,
            num_proj=8,
            forget_bias=0,
            name="rnn3"),
        tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, forget_bias=0, name="rnn4")
    ])

def buildModel(lstm_layer, is_dynamic_rnn):
    # Weights and biases for output softmax layer.
    out_weights = tf.Variable(
        tf.random_normal([num_units, n_classes]))
    out_bias = tf.Variable(tf.random_normal([n_classes]))

    # input image placeholder
    x = tf.placeholder(
        "float", [None, time_steps, n_input], name="INPUT_IMAGE")

    # x is shaped [batch_size,time_steps,num_inputs]
    if is_dynamic_rnn:
        lstm_input = tf.transpose(x, perm=[1, 0, 2])
        outputs, _ = tf.lite.experimental.nn.dynamic_rnn(
          lstm_layer, lstm_input, dtype="float32")
        outputs = tf.unstack(outputs, axis=0)
    else:
        lstm_input = tf.unstack(x, time_steps, 1)
        outputs, _ = tf.nn.static_rnn(lstm_layer, lstm_input, dtype="float32")

    # Compute logits by multiplying outputs[-1] of shape [batch_size,num_units]
    # by the softmax layer's out_weight of shape [num_units,n_classes]
    # plus out_bias
    prediction = tf.matmul(outputs[-1], out_weights) + out_bias
    output_class = tf.nn.softmax(prediction, name="OUTPUT_CLASS")

    return x, prediction, output_class


### Utility functions for training, saving/restoring, and serving (inferencing) the model

In [5]:
def trainModel(x, prediction, output_class, sess):
    # input label placeholder
    y = tf.placeholder("float", [None, n_classes])
    # Loss function
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    # Optimization
    opt = tf.train.AdamOptimizer(
        learning_rate=learning_rate).minimize(loss)

    # Initialize variables
    init = tf.global_variables_initializer()
    sess.run(init)
    for _ in range(TRAIN_STEPS):
        batch_x, batch_y = mnist.train.next_batch(
          batch_size=batch_size, shuffle=False)

        batch_x = batch_x.reshape((batch_size, time_steps,
                                 n_input))
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

def saveAndRestoreModel(lstm_layer, sess, saver, is_dynamic_rnn):
    model_dir = 'export/dynamic_rnn'
    saver.save(sess, model_dir)

    # Reset the graph.
    tf.reset_default_graph()
    x, prediction, output_class = buildModel(lstm_layer, is_dynamic_rnn)

    new_sess = tf.Session()
    saver = tf.train.Saver()
    saver.restore(new_sess, model_dir)
    return x, prediction, output_class, new_sess

def getInferenceResult(x, output_class, sess):
    b1, _ = mnist.train.next_batch(batch_size=1)
    sample_input = np.reshape(b1, (1, time_steps, n_input))
    expected_output = sess.run(output_class, feed_dict={x: sample_input})
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        sess, sess.graph_def, [output_class.op.name])
    with open('output_graph.pb', 'wb') as f:
        f.write(frozen_graph.SerializeToString())
    return sample_input, expected_output, frozen_graph

### Create and run session

#### Trick 2:
Set `control_flow_util.ENABLE_CONTROL_FLOW_V2 = True` in order to use the `tf.lite.experimental.nn` implementations

In [6]:
control_flow_util.ENABLE_CONTROL_FLOW_V2 = True

In [7]:
sess = tf.Session()
x, prediction, output_class = buildModel(
    buildLstmLayer(), is_dynamic_rnn=True)
trainModel(x, prediction, output_class, sess)

saver = tf.train.Saver()

x, prediction, output_class, new_sess = saveAndRestoreModel(
    buildLstmLayer(), sess, saver, is_dynamic_rnn=True)

test_inputs, expected_output, frozen_graph = getInferenceResult(
    x, output_class, new_sess)

### Functions for converting graph to TFLite

#### Trick 3:
Must use `convert_op_hints_to_stubs` to wrap a subpart of a TensorFlow execution graph to a single TensorFlow Lite operator.

In [8]:
def tfliteInvoke(graph, test_inputs, outputs, op_hints_to_stubs=True):
    tf.reset_default_graph()
    # Turn the input into placeholder of shape 1
    tflite_input = tf.placeholder(
        "float", [1, time_steps, n_input], name="INPUT_IMAGE_LITE")
    tf.import_graph_def(graph, name="", input_map={"INPUT_IMAGE": tflite_input})
    with tf.Session() as sess:
        curr = sess.graph_def
        if op_hints_to_stubs is True:
            curr = convert_op_hints_to_stubs(graph_def=curr)

    converter = tf.lite.TFLiteConverter(curr, [tflite_input], [outputs])
    tflite = converter.convert()
    interpreter = tf.lite.Interpreter(model_content=tflite)

    try:
        interpreter.allocate_tensors()
    except ValueError:
        print('false')
        assert False

    input_index = (interpreter.get_input_details()[0]["index"])
    interpreter.set_tensor(input_index, test_inputs)
    interpreter.invoke()
    output_index = (interpreter.get_output_details()[0]["index"])
    result = interpreter.get_tensor(output_index)
    # Reset all variables so it will not pollute other inferences.
    interpreter.reset_all_variables()
    return result

### Convert to TFLite and compare result with inference with original graph

In [9]:
result = tfliteInvoke(frozen_graph, test_inputs, output_class)
print(np.allclose(expected_output, result, rtol=1e-6, atol=1e-2))

True


We will get some unsupported operation error if we did not appply `convert_op_hints_to_stubs` to the graph

In [10]:
result = tfliteInvoke(frozen_graph, test_inputs, output_class, op_hints_to_stubs=False)

ConverterError: TOCO failed. See console for info.
2019-03-06 18:18:19.683408: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-03-06 18:18:19.706735: I tensorflow/lite/toco/import_tensorflow.cc:1335] Converting unsupported operation: TensorListFromTensor
2019-03-06 18:18:19.706780: I tensorflow/lite/toco/import_tensorflow.cc:193] Unsupported data type in placeholder op: 21
2019-03-06 18:18:19.706826: I tensorflow/lite/toco/import_tensorflow.cc:1335] Converting unsupported operation: TensorListReserve
2019-03-06 18:18:19.706855: I tensorflow/lite/toco/import_tensorflow.cc:193] Unsupported data type in placeholder op: 21
2019-03-06 18:18:19.706953: I tensorflow/lite/toco/import_tensorflow.cc:1335] Converting unsupported operation: While
2019-03-06 18:18:19.707012: I tensorflow/lite/toco/import_tensorflow.cc:193] Unsupported data type in placeholder op: 21
2019-03-06 18:18:19.707018: I t